In [1]:
import json
import os

from collections import defaultdict

import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy import create_engine

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
import nltk
import re

from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.decomposition import LatentDirichletAllocation as LDA

from pyLDAvis import sklearn as sklearn_lda
import pickle 
import pyLDAvis

In [1]:
#pip install pyLDAvis

In [3]:
engine = create_engine('postgresql+psycopg2://postgres:@localhost/stats170b')

In [3]:
bio_path = 'C:\\Users\\rtg90\\Desktop\\Course Material\\Stats 170B\\CORD-19-research-challenge\\biorxiv_medrxiv\\biorxiv_medrxiv\\pdf_json'
bio_doc_list = []

for filename in os.listdir(bio_path):
    with open(bio_path + "\\" + filename) as f:
        bio_doc_list.append(json.load(f))

In [4]:
comm_path = 'C:\\Users\\rtg90\\Desktop\\Course Material\\Stats 170B\\CORD-19-research-challenge\\comm_use_subset\\comm_use_subset\\pdf_json'
comm_doc_list = []

for filename in os.listdir(comm_path):
    with open(comm_path + "\\" + filename) as f:
        comm_doc_list.append(json.load(f))

In [5]:
cus_path = 'C:\\Users\\rtg90\\Desktop\\Course Material\\Stats 170B\\CORD-19-research-challenge\\custom_license\\custom_license\\pdf_json'
cus_doc_list = []

for filename in os.listdir(cus_path):
    with open(cus_path + "\\" + filename) as f:
        cus_doc_list.append(json.load(f))

In [6]:
noncomm_path = 'C:\\Users\\rtg90\\Desktop\\Course Material\\Stats 170B\\CORD-19-research-challenge\\noncomm_use_subset\\noncomm_use_subset\\pdf_json'
noncomm_doc_list = []

for filename in os.listdir(noncomm_path):
    with open(noncomm_path + "\\" + filename) as f:
        noncomm_doc_list.append(json.load(f))

In [7]:
article_list = []
#article_dict["bio"] = dict()
#article_dict["comm"] = dict()
#article_dict["cus"] = dict()
#article_dict["noncomm"] = dict()

In [7]:
print(len(bio_doc_list), len(comm_doc_list), len(cus_doc_list), len(noncomm_doc_list)) #ORIGINAL JSON FILE SIZE

1053 9315 20657 2350


In [8]:
print(len(bio_doc_list), len(comm_doc_list), len(cus_doc_list), len(noncomm_doc_list)) #NEW JSON FILE SIZE

1625 9524 26505 2490


In [40]:
print(bio_doc_list[0]['abstract'][0]['text'])

word count: 194 22 Text word count: 5168 23 24 25 author/funder. All rights reserved. No reuse allowed without permission. Abstract 27 The positive stranded RNA genomes of picornaviruses comprise a single large open reading 28 frame flanked by 5′ and 3′ untranslated regions (UTRs). Foot-and-mouth disease virus (FMDV) 29 has an unusually large 5′ UTR (1.3 kb) containing five structural domains. These include the 30 internal ribosome entry site (IRES), which facilitates initiation of translation, and the cis-acting 31 replication element (cre). Less well characterised structures are a 5′ terminal 360 nucleotide 32 stem-loop, a variable length poly-C-tract of approximately 100-200 nucleotides and a series of 33 two to four tandemly repeated pseudoknots (PKs). We investigated the structures of the PKs 34 by selective 2′ hydroxyl acetylation analysed by primer extension (SHAPE) analysis and 35 determined their contribution to genome replication by mutation and deletion experiments. 36 SHAPE

In [54]:
print(bio_doc_list[0]['body_text'][0]['text'])

VP3, and VP0 (which is further processed to VP2 and VP4 during virus assembly) (6). The P2 64 and P3 regions encode the non-structural proteins 2B and 2C and 3A, 3B (1-3) (VPg), 3C pro and 4 structural protein-coding region is replaced by reporter genes, allow the study of genome 68 replication without the requirement for high containment (9, 10) ( figure 1A ).


In [8]:
print(article_dict)

{'bio': {}, 'comm': {}, 'cus': {}, 'noncomm': {}}


In [9]:
source_type = [("bio", bio_doc_list), ("comm", comm_doc_list), ("cus", cus_doc_list), ("noncomm", noncomm_doc_list)]

In [12]:
'''
def abstract_generator(source_tuple, article_dict):
    for article in source_tuple[1]:
    
        for abs in article['abstract']:
            if article['paper_id'] not in article_dict[source_tuple[0]]:
                article_dict[source_tuple[0]][article['paper_id']] = defaultdict(str)
                article_dict[source_tuple[0]][article['paper_id']]['abstract'] = abs['text']
            else:
                article_dict[source_tuple[0]][article['paper_id']]['abstract'] = article_dict[source_tuple[0]][article['paper_id']]['abstract'] + " "+ abs['text']
    
    return article_dict
    
    #print(len(article['abstract']))
    
for tup in source_type:
    article_dict = abstract_generator(tup, article_dict)
'''


article_dict: {source_type: {paper_id: {abstract : text_concatenated} } } 

In [10]:
def abstract_generator(source_tuple, article_list):
    for article in source_tuple[1]:
        temp_dict = dict()
        temp_dict['id'] = article['paper_id']
        temp_dict['type'] = source_tuple[0]
        temp_dict['title'] = article['metadata']['title']
        temp_dict['abstract'] = ""
        temp_dict['bodytext'] = ""

        for abs in article['abstract']:
            temp_dict['abstract'] = temp_dict['abstract'] + " " + abs['text']
        
        for body in article['body_text']:
            temp_dict['bodytext'] = temp_dict['bodytext'] + " " + body['text']
        
        article_list.append(temp_dict)

    return article_list
    
    #print(len(article['abstract']))

In [11]:
for tup in source_type:
    article_list = abstract_generator(tup, article_list)

In [12]:
print(len(article_list))

40144


In [31]:
print(len(article_dict['bio']) + len(article_dict['comm']) + len(article_dict['noncomm']) + len(article_dict['cus'])) #ORIGINAL FILE

24827


In [14]:
print(len(bio_doc_list) + len(comm_doc_list) + len(cus_doc_list) + len(noncomm_doc_list)) #NEW FILE

40144


In [15]:
article_df = pd.DataFrame(article_list)
article_df.head()

,abstract,bodytext,id,title,type
0,word count: 194 22 Text word count: 5168 23 2...,"VP3, and VP0 (which is further processed to V...",0015023cc06b5362d332b3baf348d11567ca2fbb,The RNA pseudoknots in foot-and-mouth disease ...,bio
1,"During the past three months, a new coronavir...","In December 2019, a novel coronavirus, SARS-C...",00340eea543336d54adda18236424de6a5e91c9d,Analysis Title: Regaining perspective on SARS-...,bio
2,,The 2019-nCoV epidemic has spread across Chin...,004f0f8bb66cf446678dc13cf2701feec4f36d76,Healthcare-resource-adjusted vulnerabilities t...,bio
3,The fast accumulation of viral metagenomic da...,"Metagenomic sequencing, which allows us to di...",00911cf4f99a3d5ae5e5b787675646a743574496,CHEER: hierarCHical taxonomic classification f...,bio
4,Infectious bronchitis (IB) causes significant...,"Infectious bronchitis (IB), which is caused b...",00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,"Real-time, MinION-based, amplicon sequencing f...",bio


In [18]:
article_df.to_sql('articles', con=engine, index = False, if_exists='replace')

# RUN FROM HERE

In [4]:
article_df = pd.read_sql_query('SELECT * from articles',con=engine)
article_df.head()

,abstract,bodytext,id,title,type,topic,topic_str
0,word count: 194 22 Text word count: 5168 23 2...,"VP3, and VP0 (which is further processed to V...",0015023cc06b5362d332b3baf348d11567ca2fbb,The RNA pseudoknots in foot-and-mouth disease ...,bio,1,rna sequence gene sequences pcr using genome d...
1,"During the past three months, a new coronavir...","In December 2019, a novel coronavirus, SARS-C...",00340eea543336d54adda18236424de6a5e91c9d,Analysis Title: Regaining perspective on SARS-...,bio,2,data model number time used using 10 rate case...
2,,The 2019-nCoV epidemic has spread across Chin...,004f0f8bb66cf446678dc13cf2701feec4f36d76,Healthcare-resource-adjusted vulnerabilities t...,bio,2,data model number time used using 10 rate case...
3,The fast accumulation of viral metagenomic da...,"Metagenomic sequencing, which allows us to di...",00911cf4f99a3d5ae5e5b787675646a743574496,CHEER: hierarCHical taxonomic classification f...,bio,2,data model number time used using 10 rate case...
4,Infectious bronchitis (IB) causes significant...,"Infectious bronchitis (IB), which is caused b...",00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,"Real-time, MinION-based, amplicon sequencing f...",bio,1,rna sequence gene sequences pcr using genome d...


In [9]:
print(len(article_df))
temp_set = set()
for id in article_df['id']:
    temp_set.add(id)
    
print(len(temp_set))
#ALL ARTICLES ARE UNIQUE

40144
40144


In [17]:
title_abs = []

In [18]:
for i in range(len(article_df)):
    temp_str = article_df.iloc[i]['title'] + "\n" + article_df.iloc[i]['abstract']
    title_abs.append(temp_str)

In [19]:
only_abs = []

In [20]:
for i in range(len(article_df)):
    temp_str = article_df.iloc[i]['abstract']
    only_abs.append(temp_str)

In [10]:
stemmer = SnowballStemmer("english")
stopwords = nltk.corpus.stopwords.words('english')

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token) and len(token) > 1 and token not in stopwords:
            filtered_tokens.append(token)
    #print(filtered_tokens)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    #print(stems)
    return stems

tokenize_and_stem("Obviously, 'd aa z this d. ' ` ,'")

['obvious', "'d", 'aa', 'd.']

In [67]:
vectorizer = TfidfVectorizer(max_df=0.80, min_df = 0.05, max_features = 10000, tokenizer = tokenize_and_stem, ngram_range = (1,3))
vectors = vectorizer.fit_transform(title_abs)

feature_names = vectorizer.get_feature_names()

dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [69]:
df.head()

,'s,acid,activ,acut,acut respiratori,acut respiratori syndrom,addit,affect,age,agent,...,vaccin,various,viral,viral infect,virus,vitro,well,within,without,year
0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.193800,0.0,0.078140,0.000000,0.187345,0.0
1,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.286303,0.0,0.000000,...,0.000000,0.0,0.094457,0.0,0.000000,0.0,0.123811,0.000000,0.000000,0.0
2,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0
3,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.272689,0.0,0.073874,0.0,0.000000,0.000000,0.000000,0.0
4,0.02239,0.021834,0.0,0.0,0.0,0.0,0.077873,0.000000,0.0,0.022322,...,0.079532,0.0,0.030010,0.0,0.097560,0.0,0.000000,0.064031,0.117887,0.0


In [11]:
cnt =0
for x in df.columns:
    try:
        int(x)
        cnt += 1
    except:
        pass

NameError: name 'df' is not defined

# Topic Modelling (LDA)

In [7]:
number_topics = 10
number_words = 10

In [9]:
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
count_data = count_vectorizer.fit_transform(article_df['bodytext'])

In [34]:
#lda = LDA(n_components=number_topics, learning_method='online', n_jobs=-1)
#lda.fit(count_data)

In [26]:
def print_topics(model, count_vectorizer, n_top_words):
    top_topics = []
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        topic = " ".join([words[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        top_topics.append(topic)
        print(topic)
        
    return top_topics

print("Topics found via LDA:")
top_topics = print_topics(lda, count_vectorizer, number_words)

Topics found via LDA:

Topic #0:
health public care risk disease research information countries use diseases

Topic #1:
rna sequence gene sequences pcr using genome dna genes protein

Topic #2:
data model number time used using 10 rate cases different

Topic #3:
cells cell mice immune expression il response ifn infection induced

Topic #4:
la en des les el et le que los se

Topic #5:
cells 10 virus cell fig using protein ml infected infection

Topic #6:
patients study respiratory infection clinical children patient treatment cases infections

Topic #7:
virus al et viruses infection human influenza viral vaccine disease

Topic #8:
disease animals cats dogs blood signs lesions calves clinical diarrhea

Topic #9:
et al protein proteins binding activity membrane cell structure acid


In [17]:
#pickle.dump(lda, open('initial_lda_model.pk', 'wb'))

In [5]:
lda = pickle.load(open('initial_lda_model.pk', 'rb'))

In [ ]:
doc_topic = lda.transform(count_data)

In [44]:
topic_list = []
topic_str_list = []

for i in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[i].argmax()
    topic_list.append(topic_most_pr)    
    
    topic_str = top_topics[topic_most_pr]
    topic_str_list.append(topic_str)

article_df['topic'] = topic_list
article_df['topic_str'] = topic_str_list

In [40]:
article_df.head()

,abstract,bodytext,id,title,type,topic,topic_str
0,word count: 194 22 Text word count: 5168 23 2...,"VP3, and VP0 (which is further processed to V...",0015023cc06b5362d332b3baf348d11567ca2fbb,The RNA pseudoknots in foot-and-mouth disease ...,bio,1,rna sequence gene sequences pcr using genome d...
1,"During the past three months, a new coronavir...","In December 2019, a novel coronavirus, SARS-C...",00340eea543336d54adda18236424de6a5e91c9d,Analysis Title: Regaining perspective on SARS-...,bio,2,data model number time used using 10 rate case...
2,,The 2019-nCoV epidemic has spread across Chin...,004f0f8bb66cf446678dc13cf2701feec4f36d76,Healthcare-resource-adjusted vulnerabilities t...,bio,2,data model number time used using 10 rate case...
3,The fast accumulation of viral metagenomic da...,"Metagenomic sequencing, which allows us to di...",00911cf4f99a3d5ae5e5b787675646a743574496,CHEER: hierarCHical taxonomic classification f...,bio,2,data model number time used using 10 rate case...
4,Infectious bronchitis (IB) causes significant...,"Infectious bronchitis (IB), which is caused b...",00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,"Real-time, MinION-based, amplicon sequencing f...",bio,1,rna sequence gene sequences pcr using genome d...


In [41]:
len(article_df[article_df['topic'] == 6])

6616

In [42]:
len(article_df[article_df['topic'] == 0])

5398

In [43]:
article_df.shape

(40144, 7)

In [ ]:
LDAvis_data_filepath = os.path.join('./ldavis_prepared_initial_'+ str(number_topics))

In [2]:
LDAvis_prepared = sklearn_lda.prepare(lda, count_data, count_vectorizer)
#with open(LDAvis_data_filepath, 'w') as f:
#        pickle.dump(LDAvis_prepared, f)


In [20]:
#pickle.dump(LDAvis_prepared, open(LDAvis_data_filepath, 'wb'))
        
        

# load the pre-prepared pyLDAvis data from disk
#with open(LDAvis_data_filepath) as f:
#    LDAvis_prepared = pickle.load(f)
LDAvis_prepared = pickle.load(open(LDAvis_data_filepath, 'rb'))

    
#pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_initial_'+ str(number_topics) +'.html')

In [31]:
pyLDAvis.display(LDAvis_prepared)